In [ ]:
%pip install Faker
import re
import random
from typing import List
import numpy as np
import pandas as pd
from faker import Faker

def gen_customers(
    n_unique: int = 500,
    dup_rate: float = 0.30,
    max_dups_per: int = 2,
    locale: str = "en_US",
    seed: int = 42
) -> pd.DataFrame:
    """
    Генерирует таблицу клиентов с дубликатами.
    - n_unique: колич. уникальных клиентов (без дублей)
    - dup_rate: доля уникальных клиентов, которым добавим дубликаты
    - max_dups_per: максимум дублей на одного клиента (1..2 обычно достаточно)
    - locale: язык Faker (en_US, de_DE и т.п.)
    - seed: фиксируем случайность для воспроизводимости
    Возвращает DataFrame со столбцами:
    uid (истинная личность), name, street, city, zip, email, phone
    """
    rng = random.Random(seed)
    np.random.seed(seed)
    fake = Faker(locale)
    Faker.seed(seed)

    def only_digits(s: str, n: int = 5) -> str:
        d = "".join(ch for ch in s if ch.isdigit())
        return (d + "0"*n)[:n] if n else d

    def noisy_str(s: str) -> str:
        """Вносит лёгкий шум: удаление/перестановка/замена символов."""
        if not s or len(s) < 2:
            return s
        s_list = list(s)
        # замена символа
        if rng.random() < 0.35:
            i = rng.randrange(len(s_list))
            if s_list[i].isalpha():
                s_list[i] = chr(((ord(s_list[i].lower()) - 97 + 1) % 26) + 97)
        # удаление символа
        if rng.random() < 0.25 and len(s_list) > 1:
            i = rng.randrange(len(s_list))
            s_list.pop(i)
        # перестановка соседних
        if rng.random() < 0.25 and len(s_list) > 2:
            i = rng.randrange(len(s_list) - 1)
            s_list[i], s_list[i+1] = s_list[i+1], s_list[i]
        return "".join(s_list)

    def tweak_email(email: str) -> str:
        if rng.random() < 0.6:
            # лёгкая опечатка в локальной части
            local, domain = email.split("@")
            if len(local) > 3:
                i = rng.randrange(len(local))
                local = local[:i] + "" + local[i+1:]
            email = f"{local}@{domain}"
        if rng.random() < 0.3:
            email = email.replace(".com", ".co", 1)
        return email

    def tweak_phone(phone: str) -> str:
        # заменим 1–2 цифры
        phone = list(phone)
        for _ in range(rng.randint(0, 2)):
            i = rng.randrange(len(phone))
            if phone[i].isdigit():
                phone[i] = str(rng.randrange(10))
        return "".join(phone)

    rows: List[dict] = []
    uid = 0
    for _ in range(n_unique):
        uid += 1
        name = fake.name()
        street = f"{fake.street_name()} {rng.randint(1, 199)}"
        city = fake.city()
        zipc = only_digits(fake.postcode(), 5)
        email_local = re.sub(r"[^a-z0-9]+", ".", name.lower())
        email = f"{email_local}@example.com"
        phone = "0" + "".join(str(rng.randrange(10)) for _ in range(9))

        base = dict(uid=uid, name=name, street=street, city=city, zip=zipc, email=email, phone=phone)
        rows.append(base)

        # создаём дубли с вероятностью dup_rate
        if rng.random() < dup_rate:
            for _ in range(rng.randint(1, max_dups_per)):
                rows.append(dict(
                    uid=uid,
                    name=noisy_str(name),
                    street=noisy_str(street),
                    city=city if rng.random() < 0.9 else noisy_str(city),
                    zip=zipc,  # индекс часто совпадает
                    email=email if rng.random() < 0.7 else tweak_email(email),
                    phone=phone if rng.random() < 0.7 else tweak_phone(phone),
                ))

    df = pd.DataFrame(rows).reset_index(drop=True)
    # полезные служебные поля
    df.insert(0, "row_id", np.arange(1, len(df) + 1))
    return df

# Пример использования:
if __name__ == "__main__":
    df = gen_customers(n_unique=500, dup_rate=0.3, max_dups_per=2, locale="en_US", seed=42)
    print(df.head(10))
    print("\nВсего строк:", len(df), " | Уникальных клиентов (uid):", df["uid"].nunique())
    # Сохраняем в CSV
    df.to_csv("data/customers_synthetic.csv", index=False)
    print('\nФайл сохранён: customers_synthetic.csv')


Note: you may need to restart the kernel to use updated packages.
   row_id               name               street             city    zip  \
0       1       Allison Hill      Donald Cove 164   New Roberttown  12781   
1       2   Jonathan Johnson   Jennifer Squares 9    Robinsonshire  36964   
2       3    Abigail Shaffer  Peterson Drives 180     Port Matthew  12657   
3       4         Ian Cooper      Roman Stream 40      Herrerafurt  10829   
4       5         Ian Cooper      Roman Stream 40      Herrerafurt  10829   
5       6  Sandra Montgomery       Ray Squares 75  North Donnaport  10959   
6       7       Sharon James       Reid Lakes 172     West Michael  88342   
7       8       Sharon James       seid Lakes 172     West Michael  88342   
8       9       Timothy Wong   Amanda Gardens 102       South Noah  84387   
9      10     Colleen Nguyen    Jeremy Bypass 150      Richardland  28154   

                           email       phone  
0       allison.hill@example.com  01043